In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
df = spark.read.format("parquet").load("abfss://bronze@stretaildataproj.dfs.core.windows.net/customers/")
df.display()

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.display()

In [0]:
df = df.withColumn("domains", split(col("email"), "@")[1])
display(df)

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers" , ascending=False).display()

In [0]:
df_extract_gmail = df.filter(col("domains") == 'gmail.com')
df_extract_gmail.display()

df_extract_hotmail = df.filter(col("domains") == 'hotmail.com')
df_extract_hotmail.display()


df_extract_yahoo = df.filter(col("domains") == 'yahoo.com')
df_extract_yahoo.display()

In [0]:
df = df.withColumn("full_name" ,concat(col("first_name"),lit(' '),col("last_name")))
df = df.drop("first_name","last_name")
df.display()

### Data Writing Back  To Silver's Customer Folder

In [0]:
df.write.mode("append").format("delta").save("abfss://silver@stretaildataproj.dfs.core.windows.net/customers/")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS retail_databricks_catalog.silver_schema.customer_silver
USING DELTA
LOCATION 'abfss://silver@stretaildataproj.dfs.core.windows.net/customers/'
  


In [0]:
%sql
select * from retail_databricks_catalog.silver_schema.customer_silver